In [ ]:
!nvidia-smi

In [ ]:
import os
import torch
import sys
import platform

# 1. Clone Repository (Safe to re-run)
if not os.path.exists("InfiniteTalk"):
    !git clone https://github.com/MeiGen-AI/InfiniteTalk
else:
    print("Repository already cloned.")

# 2. Enter Directory (Required after every restart)
%cd InfiniteTalk

# 3. Safe Dependency Installation
py_ver = f"cp{sys.version_info.major}{sys.version_info.minor}"
torch_ver = torch.__version__.split('+')[0]
torch_ver_short = ".".join(torch_ver.split(".")[:2])
cuda_ver = torch.version.cuda.replace('.', '')[:3] if torch.version.cuda else "cpu"
abi = "TRUE" if torch._C._GLIBCXX_USE_CXX11_ABI else "FALSE"

print(f"✅ Detected: Python={py_ver}, Torch={torch_ver}, CUDA={cuda_ver}, ABI={abi}")
print("Installing dependencies... (Fast mode enabled)")

# Install dependencies needed for extensions
!pip install --no-cache-dir packaging ninja psutil wheel

# Install Flash Attention 2 (Dynamic Wheel Selection)
try:
    import flash_attn
    print("✅ Flash Attention already installed.")
except ImportError:
    print("Installing Flash Attention...")
    # We try 2.8.3 (Best for Torch 2.9/2.10) then 2.7.4.post1 (Fallback)
    success = False
    # Common CUDA tags in wheels are often 'cu12' for any 12.x or 'cu130' for 13.0
    cu_tags = [f"cu{cuda_ver[:2]}", f"cu{cuda_ver}", "cu12", "cu130"]
    
    for fa_v in ["2.8.3", "2.7.4.post1"]:
        if success: break
        for cu_tag in cu_tags:
            wheel = f"flash_attn-{fa_v}+{cu_tag}torch{torch_ver_short}cxx11abi{abi}-{py_ver}-{py_ver}-linux_x86_64.whl"
            # Release URLs usually don't have .post1 in the tag but do in the filename
            release_tag = f"v{fa_v.split('.post')[0]}"
            url = f"https://github.com/Dao-AILab/flash-attention/releases/download/{release_tag}/{wheel}"
            
            print(f"Trying: {url}")
            res = os.system(f"pip install --no-cache-dir {url}")
            if res == 0:
                print(f"✅ Successfully installed Flash Attention {fa_v} via wheel")
                success = True
                break
    
    if not success:
        print("⚠️ No matching wheel found. Falling back to slow source build...")
        !pip install flash-attn --no-build-isolation

# Install other dependencies from requirements.txt
!sed -i 's/numpy>=1.23.5,<2/numpy>=1.23.5/' requirements.txt
!pip install --no-cache-dir -r requirements.txt
!pip install --no-cache-dir librosa huggingface_hub

# Install system dependencies
!apt-get update && apt-get install -y ffmpeg > /dev/null 2>&1

print("\n✅ Setup Complete! If you see a 'Restart Session' button, you can IGNORE it and try running the next cell.")

In [ ]:
import os
os.makedirs("weights", exist_ok=True)

# Download base model and InfiniteTalk weights
!hf download Wan-AI/Wan2.1-I2V-14B-480P --local-dir ./weights/Wan2.1-I2V-14B-480P
!hf download MeiGen-AI/InfiniteTalk --local-dir ./weights/InfiniteTalk

# Download English audio encoder (DEFAULT)
!hf download facebook/wav2vec2-base-960h --local-dir ./weights/wav2vec2-base-960h

# Alternative: For Chinese audio support, uncomment the lines below:
# !hf download TencentGameMate/chinese-wav2vec2-base --local-dir ./weights/chinese-wav2vec2-base
# !hf download TencentGameMate/chinese-wav2vec2-base model.safetensors --revision refs/pr/1 --local-dir ./weights/chinese-wav2vec2-base

In [ ]:
# Enable public link for Gradio
!sed -i 's/demo.launch(server_name="0.0.0.0", debug=True, server_port=8418)/demo.launch(server_name="0.0.0.0", debug=True, server_port=8418, share=True)/' app.py

# Launch Gradio interface with English audio encoder (DEFAULT)
!python app.py --ckpt_dir weights/Wan2.1-I2V-14B-480P --wav2vec_dir 'weights/wav2vec2-base-960h' --infinitetalk_dir weights/InfiniteTalk/single/infinitetalk.safetensors --num_persistent_param_in_dit 0 --motion_frame 9

# Alternative: For Chinese audio, uncomment below and comment the English line above:
# !python app.py --ckpt_dir weights/Wan2.1-I2V-14B-480P --wav2vec_dir 'weights/chinese-wav2vec2-base' --infinitetalk_dir weights/InfiniteTalk/single/infinitetalk.safetensors --num_persistent_param_in_dit 0 --motion_frame 9